In [35]:
import os
import pandas as pd
import ast
# set the path to the top-level directory containing the multirun folders
top_dir = 'multirun/'

# define a lambda function to convert the rewards column to a nested list of integers
rewards_converter = lambda x: [[int(val) for val in row[1:-1].split()] for row in x.split('\n') if row]

# initialize an empty list to hold the dataframes
df_list = []

# traverse the directory structure and load the rewards.csv files
for date_folder in os.listdir(top_dir):
    date_path = os.path.join(top_dir, date_folder)
    if not os.path.isdir(date_path):
        continue
    for time_folder in os.listdir(date_path):
        time_path = os.path.join(date_path, time_folder)
        if not os.path.isdir(time_path):
            continue
        for exp_folder in os.listdir(time_path):
            exp_path = os.path.join(time_path, exp_folder)
            if not os.path.isdir(exp_path):
                continue
            reward_file = os.path.join(exp_path, 'rewards.csv')
            if not os.path.isfile(reward_file):
                continue
            # load the rewards.csv file and add additional columns
            #converters = {'rewards': lambda x: ast.literal_eval(x) if isinstance(x, str) else None}
            try:
                df = pd.read_csv(reward_file)#, converters=converters)
            except Exception as e:
                print(f"Error loading {reward_file}: {e}")
                continue
            
            rewards = [float(t.strip('[ ]')) for t in df['rewards'][0].split('\n')]
            for i, reward in enumerate(rewards):
                df[f'reward_trial{i}'] = reward
            
            df.drop(columns=['rewards'], inplace=True)
            df['date'] = date_folder
            df['time'] = time_folder
            df['experiment'] = exp_folder
            df_list.append(df)

# concatenate the dataframes into one
result_df = pd.concat(df_list, ignore_index=True)


In [37]:


result_df = result_df[result_df['date'] == '2023-05-04' ]

result_df['time'] = pd.to_datetime(result_df['time'], format='%H-%M-%S')
df_filtered = result_df[result_df['time'] > pd.to_datetime('12:00:00', format='%H:%M:%S')]

#select columns with time after 12:00
#result_df = result_df[result_df['time'] > '12:00:00' ]
result_df


,optimizer,seed,phys_nn_config,reward_trial0,reward_trial1,reward_trial2,reward_trial3,reward_trial4,reward_trial5,reward_trial6,reward_trial7,reward_trial8,reward_trial9,reward_trial10,date,time,experiment
0,CEM,1,0,0.0,98.0,152.0,164.0,200.0,200.0,200.0,200.0,200.0,200.0,200.0,2023-05-04,1900-01-01 04:46:01,0
1,CEM,1,0,0.0,79.0,200.0,183.0,200.0,200.0,200.0,200.0,200.0,200.0,200.0,2023-05-04,1900-01-01 04:46:01,1
2,ICEM,1,1,0.0,55.0,200.0,200.0,160.0,200.0,200.0,200.0,200.0,200.0,200.0,2023-05-04,1900-01-01 12:06:58,3
3,ICEM,1,0,0.0,55.0,166.0,200.0,200.0,200.0,200.0,200.0,200.0,200.0,200.0,2023-05-04,1900-01-01 12:06:58,0
4,ICEM,1,2,0.0,185.0,136.0,200.0,200.0,200.0,200.0,200.0,200.0,200.0,200.0,2023-05-04,1900-01-01 12:06:58,5
5,ICEM,1,2,0.0,109.0,200.0,200.0,200.0,200.0,200.0,200.0,200.0,200.0,200.0,2023-05-04,1900-01-01 12:06:58,4
6,ICEM,1,1,0.0,189.0,200.0,200.0,200.0,200.0,200.0,200.0,200.0,200.0,200.0,2023-05-04,1900-01-01 12:06:58,2
7,ICEM,1,3,0.0,200.0,200.0,200.0,200.0,200.0,200.0,200.0,200.0,200.0,200.0,2023-05-04,1900-01-01 12:06:58,7
8,ICEM,1,3,0.0,200.0,200.0,200.0,147.0,104.0,200.0,200.0,200.0,200.0,200.0,2023-05-04,1900-01-01 12:06:58,6
9,ICEM,1,0,0.0,200.0,200.0,200.0,200.0,200.0,200.0,200.0,200.0,200.0,200.0,2023-05-04,1900-01-01 12:06:58,1
